In [1]:
! pip install nose
import numpy as np
import nose.tools as nt
from collections import defaultdict
import pandas as pd
from pandas import *
from copy import deepcopy
import matplotlib.pyplot as plt

     |████████████████████████████████| 154 kB 9.2 MB/s 


In [2]:
matchPairings = defaultdict(lambda: defaultdict(dict))
matchPairings["A"]["U"] = 1
matchPairings["U"]["A"] = 1
matchPairings["G"]["C"] = 1
matchPairings["C"]["G"] = 1
# Take in a sequence, output DP array
def nussinovDP(sequence):
  DP = np.zeros((len(sequence), len(sequence)))
  for diag in range(1,len(sequence)):
    for i in range(diag, len(sequence)):
      j = i-diag
      if matchPairings[sequence[i]][sequence[j]] == 1:
        DP[j][i] = max(DP[j+1][i-1] + 1, DP[j][i-1], DP[j+1][i])
      else:
        DP[j][i] = max(DP[j][i-1], DP[j+1][i], DP[j+1][i-1])
      for k in range(j+1, i):
        DP[j][i] = max(DP[j][i], DP[j][k]+DP[k+1][i])
  return DataFrame(DP)
# Use DP array, backtrace and enumerate possible solutions

In [3]:
# Test Cases
test1 = nussinovDP("GGGAAAUCC")
nt.assert_equals(test1[8][0],3)
print(test1)
test2 = nussinovDP("AUGCGCGGGAUCGUCGAGAU")
print(test2)
nt.assert_equals(test2[19][0],8)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Please use assertEqual instead.
  This is separate from the ipykernel package so we can avoid doing imports until


     0    1    2    3    4    5    6    7    8
0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  2.0  3.0
1  0.0  0.0  0.0  0.0  0.0  0.0  1.0  2.0  3.0
2  0.0  0.0  0.0  0.0  0.0  0.0  1.0  2.0  2.0
3  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0
4  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0
5  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0
6  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
7  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
8  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0    1    2    3    4    5    6   ...   13   14   15   16   17   18   19
0   0.0  1.0  1.0  2.0  2.0  3.0  3.0  ...  5.0  6.0  6.0  7.0  7.0  7.0  8.0
1   0.0  0.0  0.0  1.0  1.0  2.0  2.0  ...  4.0  5.0  5.0  6.0  6.0  7.0  7.0
2   0.0  0.0  0.0  1.0  1.0  2.0  2.0  ...  4.0  5.0  5.0  6.0  6.0  6.0  7.0
3   0.0  0.0  0.0  0.0  1.0  1.0  2.0  ...  4.0  5.0  5.0  6.0  6.0  6.0  7.0
4   0.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  3.0  4.0  4.0  5.0  5.0  5.0  6.0
5   0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  3.0  4.0  4.0  5.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Please use assertEqual instead.
  import sys


In [4]:
pairs = {"A": "U", "U": "A", "G": "C", "C": "G"}

def valid_pair(pair):
  return True if pair in pairs.items() else False

def traceback(i, j, sequence, dp, matches):

  if i < j:
    if dp[i][j] == dp[i+1][j]:
      traceback(i+1, j, sequence, dp, matches)
    
    elif dp[i][j] == dp[i][j-1]:
      traceback(i, j-1, sequence, dp, matches)
    
    elif valid_pair((sequence[i], sequence[j])) and dp[i][j] == dp[i+1][j-1] + 1:
      matches.append((i, j))
      traceback(i+1, j-1, sequence, dp, matches) 

    else:
      for k in range(i+1, j):
        if dp[i][j] == dp[i][k] + dp[k+1][j]:
          traceback(i, k, sequence, dp, matches)
          traceback(k+1, j, sequence, dp, matches)
          break
    
def dot_write(rna, fold):
    dot = ["." for i in range(len(rna))]
    for s in fold:
        dot[min(s)] = "("
        dot[max(s)] = ")"
    return "".join(dot)


def test_seq(seq):
  dp = nussinovDP(seq).to_numpy()
  matches = []
  traceback(0, len(seq) - 1, seq, dp, matches)
  out = dot_write(seq, matches)
  return out


In [5]:
# Test Cases
test3 = test_seq("GGGAAAUCC")
nt.assert_equals(test3, ".((..()))")
test4 = test_seq("AUGCGCGGGAUCGUCGAGAU")
nt.assert_equals(test4,"((.()()..()()(()).))")
test5 = test_seq("GCUCGGGUUCCCUAUUCAAGAGC")
nt.assert_equals(test5,"(((((((..)))(()(.))))))")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Please use assertEqual instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Please use assertEqual instead.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Please use assertEqual instead.
  import sys


In [6]:
pairs = {"A": "U", "U": "A", "G": "C", "C": "G"}

def valid_pair(pair):
  return True if pair in pairs.items() else False

def traceback_enum(i, j, sequence, dp, currmatch, matches):
  if i < j:
    if dp[i][j] == dp[i+1][j]:
      copy = deepcopy(currmatch)
      traceback_enum(i+1, j, sequence, dp, copy, matches)
    
    if dp[i][j] == dp[i][j-1]:
      copy = deepcopy(currmatch)
      traceback_enum(i, j-1, sequence, dp, copy, matches)
    
    if valid_pair((sequence[i], sequence[j])) and dp[i][j] == dp[i+1][j-1] + 1:
      currmatch.append((i, j))
      traceback_enum(i+1, j-1, sequence, dp, currmatch, matches) 
    for k in range(i+1, j):
      if dp[i][j] == dp[i][k] + dp[k+1][j]:
        copy = deepcopy(currmatch)
        traceback_enum(i, k, sequence, dp, copy, matches)
        traceback_enum(k+1, j, sequence, dp, copy, matches)
        break
  elif len(currmatch) == int(dp[0][len(dp[0])-1]) and currmatch not in matches:
    matches.append(tuple(currmatch))


def test_seq_enum(seq):
  dp = nussinovDP(seq).to_numpy()
  currmatch = []
  matches = []
  traceback_enum(0, len(seq) - 1, seq, dp, currmatch, matches)
  print(seq)
  for match in set(matches):
    out = dot_write(seq, match)
    print(out)

test_seq_enum("AU")
test_seq_enum("AUU")
test_seq_enum("AAUU")
test_seq_enum("AAUUU")
test_seq_enum("GGGAAAUCC")
test_seq_enum("AUGCGCGGGAUCGUCGAGAU")
test_seq_enum("GCUCGGGUUCCCUAUUCAAGAGC")

AU
()
AUU
(.)
().
AAUU
(())
AAUUU
(()).
((.))
(().)
GGGAAAUCC
(.(..()))
.((.(.)))
((..(.)))
.(((..)))
((.(..)))
(.((..)))
.((..()))
((...()))
(.(.(.)))
AUGCGCGGGAUCGUCGAGAU
((()()...()(.(()))))
((().(...()()(()))))
((()()...()()(()).))
((()().....()(()))))
((.()()..()()....)))
((.()()..()()(()).))
((.(()...()()(()))))
((.()()....()(()))))
((().()..()()....)))
()()()...((().....))
()().()....()(()).))
().()()..((().....))
((().()....()(()))))
().()()....()(()).))
((().()..()()(()).))
((.......)(()(()))))
((.()()....()....)))
((.(.()..()()(()))))
((.()(.....()(()))))
()().()..((().....))
()()().....()(()).))
((().()....(.(()))))
GCUCGGGUUCCCUAUUCAAGAGC
()(((((..))).......))))
()(()((..))........))))
(((((((..)))(()(.))))))
(((()((..))(()....)))))
(((()((..))((()...)))))


In [11]:
from collections import deque
def visualizer(seq, table, matches):

    n = len(seq)
    pairs = set(matches)
    left_pointers = set([pair[0] for pair in matches])
    right_pointers = set([pair[1] for pair in matches])
    backtrace_positions = np.zeros((n, n)) # list of positions that have been backtrace_positions already
    arrows = [] # directions of the form (x, y, dx, dy)
    curr_pos = 1

    stack = deque()
    stack.append((0, n-1))
    backtrace_positions[0, n-1] = 1

    while stack:
        curr_pos += 1
        (i, j) = stack.pop()

        if i >= j:
            continue

        elif (i, j) in pairs:
            arrows.append((i, j, 1, -1))
            stack.append((i+1, j-1))
            backtrace_positions[i+1, j-1] = curr_pos

        elif i in left_pointers and j in right_pointers:
            for k in range(i+1, j):
                if (i, k) in pairs:
                    arrows.append((i, j, 0, k-j))
                    stack.append((i, k))
                    backtrace_positions[i, k] = curr_pos

                    arrows.append((i, j, k+1-i, 0))
                    stack.append((k+1, j))
                    backtrace_positions[k+1, j] = curr_pos

                    break
        elif i in left_pointers:
            arrows.append((i, j, 0, -1))
            stack.append((i, j-1))
            backtrace_positions[i, j-1] = curr_pos

        else:
            arrows.append((i, j, 1, 0))
            stack.append((i+1, j))
            backtrace_positions[i+1, j] = curr_pos

    dw = dot_write(seq, matches)
    ax = plt.gca()

    backtrace_positions[backtrace_positions > 0] = backtrace_positions.max() + 1 - backtrace_positions[backtrace_positions>0]

    im = ax.imshow(backtrace_positions, cmap='YlGn')

    threshold = im.norm(im.get_array().max())/2
    table = np.array(table)
    ax.set_xticks(np.arange(n))
    ax.set_xticklabels([dw[i] + '\n' + seq[i] for i in range(n)])
    ax.set_yticks(np.arange(n))
    ax.set_yticklabels([seq[i] for i in range(n)])

    for i in range(n):
        for j in range(n):
            ax.text(j, i, table[i, j],
                           ha='center', va='center',
                           color='r' if im.norm(backtrace_positions[i, j]) > threshold else 'black')

    for (x, y, dx, dy) in arrows:
        ax.arrow(y, x, dy, dx, width=0.01, head_width=0.2, alpha=0.5, color='r', zorder=2)

    plt.savefig('output.png')
    plt.close()

In [12]:
dp = np.array([[0,1,1,1,2,2,2,3], [0,0,0,0,1,1,1,2], [0,0,0,0,1,1,1,2], [0,0,0,0,1,1,1,2], [0,0,0,0,0,0,1,1], [0,0,0,0,0,0,0,1], [0,0,0,0,0,0,0,1], [0,0,0,0,0,0,0,0]])
matches = [(0, 1), (3, 4), (6, 7)]
# traceback(0, len("AAUU") - 1, "AAUU", dp, matches)
visualizer("GCACGACG", dp, matches)